In [1]:
import numpy as np
# The pulse simulator
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ
from qiskit.providers.aer import PulseSimulator

# Object for representing physical models
from qiskit.providers.aer.pulse import PulseSystemModel

# Mock Armonk backend
from qiskit.test.mock import FakeAthens
from qiskit.test.mock.backends.armonk.fake_armonk import FakeArmonk
from qiskit.compiler import assemble

In [51]:
IBMQ.load_account()

# provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
real_backend = provider.get_backend('ibmq_athens')

ibmqfactory.load_account:WARNING:2020-08-18 17:24:47,161: Credentials are already in use. The existing account in the session will be replaced.


In [52]:
# armonk_backend = FakeArmonk()
backend = FakeAthens()

In [53]:
# set the drive strength
# getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = 63455017.85

# backend.configuration().hamiltonian['vars'] = real_backend.configuration().hamiltonian['vars']
# backend.configuration().hamiltonian['vars']

In [54]:
num_qub=5
lst = ['omegad' + str(i) for i in range(num_qub)]
for ele in lst:
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 0.5 * 1e9 * getattr(real_backend.configuration(), 'hamiltonian')['vars'][ele]
# freq_est = 4.97e9
# armonk_backend.defaults().qubit_freq_est = [freq_est]
lst2 = ['wq' + str(i) for i in range(num_qub)]
for i,ele in enumerate(lst2):
    getattr(backend.configuration(), 'hamiltonian')['vars'][ele] = 2 * np.pi * real_backend.defaults().qubit_freq_est[i]

backend.defaults().qubit_freq_est = real_backend.defaults().qubit_freq_est

In [55]:
# backend.configuration().hamiltonian['vars']

In [56]:
# armonk_model = PulseSystemModel.from_backend(armonk_backend)
athens_model = PulseSystemModel.from_backend(backend)

In [62]:
circ = QuantumCircuit(num_qub)

circ.x(0)
circ.measure_all()

In [63]:
def_trans_circ = qiskit.transpile(circ, basis_gates=backend.configuration().basis_gates)
default_schedule = qiskit.schedule(def_trans_circ, inst_map = backend.defaults().instruction_schedule_map,
                        meas_map = backend.configuration().meas_map)

changed instruction to instruction.name


In [64]:
backend_sim = PulseSimulator(configuration=backend)

my_qobj = assemble(default_schedule,
                backend=backend_sim,
                # qubit_lo_freq=backend.defaults().qubit_freq_est,
                meas_level=2,
                meas_return='single',
                shots=2048)

In [65]:
sim_result = backend_sim.run(my_qobj, athens_model).result()

In [66]:
sim_result.get_counts()

{'00000': 1736, '00001': 312}